In [1]:
import os
import pandas as pd
os.chdir("../")
os.chdir("data")

In [2]:
import collections
from collections import OrderedDict
from matplotlib import pyplot as plt
from matplotlib import cm
import pylab
import math
import numpy as np
plt.ioff()

In [3]:
from tqdm import tqdm

In [4]:
class CGR():
    K = 0
    c = None
    h = ""
    Data = ""
    i = 0
    def __init__(self,a):
        self.i=a
        
    def read_fasta(self,loc):
        f = open(loc)
        s1 = f.read()
        data = "".join(s1.split("\n")[1:])
        head = "".join(s1.split("\n")[0:1])
        return data,head
    
    def count_kmers(self,sequence, k):
        d = collections.defaultdict(int)
        for i in range(len(self.Data)-(k-1)):
            d[sequence[i:i+k]] +=1
        d.pop("N",None)
        return d

    def probabilities(self,kmer_count, k):
        probabilities = collections.defaultdict(float)
        N = len(self.Data)
        for key, value in kmer_count.items():
            probabilities[key] = float(value) / (N - k + 1)
        return probabilities

    def chaos_game_representation(self,probabilities, k):
        array_size = int(math.sqrt(4**k))
        chaos = []
        for i in range(array_size):
            chaos.append([0]*array_size)
        maxx = array_size
        maxy = array_size
        posx = 1
        posy = 1
        for key, value in probabilities.items():
            for char in key:
                if char == "T":
                    posx +=  maxx/2
                elif char == "C":
                    posy += maxy/2
                elif char == "G":
                    posx += maxx/2
                    posy += maxy/2
                maxx /= 2
                maxy /= 2

            chaos[int(posy-1)][int(posx-1)] = value
            maxx = array_size
            maxy = array_size
            posx = 1
            posy = 1
        m = float(np.amax(chaos))
        c = np.array(chaos)/m
        return c

    def load_fasta(self,loc,k):
        data,head = self.read_fasta(loc)
        self.Data = data
        f4 = self.count_kmers(data, k)
        f4_prob = self.probabilities(f4, k)
        chaos_k4 = self.chaos_game_representation(f4_prob, k)
        self.c = chaos_k4
        self.h = head
        return chaos_k4
    
    def read(self,data,head,k):
        #data,head = self.read_fasta(loc)
        self.Data = data
        f4 = self.count_kmers(data, k)
        f4_prob = self.probabilities(f4, k)
        chaos_k4 = self.chaos_game_representation(f4_prob, k)
        self.c = chaos_k4
        self.h = head
        return chaos_k4
    
    def show(self):
        plt.figure(figsize=(12,12))
        plt.title('CGR of '+str(self.K)+'-mers for '+self.h[2:])
        plt.pcolor(self.c,cmap=cm.gray_r)#,interpolation = "spline36")
        #pylab.savefig(str(self.i)+".PNG")
        plt.show()
    
    def show_v2(self):
        fig = plt.figure(figsize=(12,12))
        plt.axis('off')
        plt.imshow(self.c, cmap=cm.gray_r)#,interpolation = "spline36")
        plt.savefig(str(self.i)+".png",bbox_inches='tight',pad_inches=0)
        plt.close(fig)

In [5]:
oslist = os.listdir()

In [7]:
for i in oslist:
    if (i.split('.')[-1]=='xlsx'):
        x = pd.read_excel(i,engine = 'openpyxl',header=None)
        acc = x[0].values.tolist()
        seq = x[1].values.tolist()
        di = i.split('.')[0]
        os.makedirs(di)
        os.chdir(di)
        for b in tqdm(range(len(acc))):
            try:
                a = CGR(acc[b].replace(',',"").replace('.',""))
                a.read(seq[b],acc[b],7)
                a.show_v2()
            except:
                continue
        os.chdir('..')

 69%|██████▉   | 20/29 [00:02<00:00,  9.28it/s]<ipython-input-4-2e028886989b>:89: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(12,12))
100%|██████████| 133/133 [00:20<00:00,  6.52it/s]
